In [31]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [8]:
## Read the pdfs from the folder
loader = PyPDFDirectoryLoader('./us_census')
documents = loader.load()
text_spillter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_document = text_spillter.split_documents(documents)
final_document[0]


Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census/acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, w

In [11]:
## Embedding Using huggingface

huggingface_embedding =HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device':'cpu'}, encode_kwargs={'normalize_embeddings': True})

In [15]:
import numpy as np
print(np.array(huggingface_embedding.embed_query(final_document[0].page_content)))
print(np.array(huggingface_embedding.embed_query(final_document[0].page_content)).shape)

[-2.49750242e-02  7.48806819e-03  6.07665628e-02  2.87433621e-02
  7.38344267e-02  1.22540519e-01 -1.49952685e-02 -7.34639540e-03
 -1.00141101e-01 -1.65566453e-03 -1.06491093e-02  9.63608921e-02
 -2.55548079e-02 -1.02058493e-01  1.60375629e-02  1.36243338e-02
  1.47681087e-02 -3.22496183e-02 -2.84116510e-02  8.82342160e-02
 -3.01535577e-02  2.01463345e-02 -5.24294935e-02 -3.65697872e-03
  1.23102041e-02  5.61479293e-03  4.09074128e-02 -1.13200012e-03
 -9.47217364e-03  6.47103339e-02  8.98134559e-02 -5.35425777e-03
  2.55072154e-02  1.17486008e-02  2.94315256e-02 -4.34862711e-02
 -3.71153466e-02  2.39202976e-02 -9.60925892e-02  1.52798463e-02
 -3.21496129e-02 -4.64301370e-02 -8.10123980e-02  1.04659721e-01
  2.16492880e-02 -1.19755242e-03 -4.37056869e-02  8.71711150e-02
 -1.37320813e-02  5.47637306e-02  7.84204167e-04  3.08593139e-02
  3.65715623e-02  1.41634336e-02  1.16748204e-02 -3.02507598e-02
 -3.20119113e-02 -2.99772602e-02 -2.04795673e-02 -1.95126503e-03
 -1.50643205e-02 -2.05804

In [16]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_document[:120],huggingface_embedding)

In [17]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [18]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x33c4eddf0> search_kwargs={'k': 3}


In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']= ''

In [22]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

/var/folders/7c/r5h316lj3fq2249w_qw64ckw0000gn/T/ipykernel_14952/386166796.py:3: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  hf=HuggingFaceHub(


AttributeError: 'InferenceClient' object has no attribute 'post'

In [23]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-v0.1.
401 Client Error. (Request ID: Root=1-6866ae8d-5b43be085cbc80491483bf67;3ee7b414-9a15-4aeb-bd90-0d1b42cd2586)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [26]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [27]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [32]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [33]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [34]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

AttributeError: 'InferenceClient' object has no attribute 'post'